In [10]:
import pandas as pd
df = pd.read_csv("./input_data_1.csv",header=None).drop(0,axis=0).drop([0,1,2,3],axis=1)
df2 = pd.read_csv("./input_data_2.csv",header=None).drop(0,axis=0).drop([0,1,2,3],axis=1)



df

,4,5,6,7,8,9,10,11,12,13,...,141,142,143,144,145,146,147,148,149,150
1,[[ 0.12480081 1.1112256 2.9385915 ... -2.6...,[[-2.924284 -2.4681082 0.16968031 ... -0.1...,[[-0.8278215 -0.58498204 3.8628337 ... 4.0...,[[ 0.4455528 -1.02631247 -1.65579915 ... -2.0...,[[ -0.6036111 0.01628295 -0.02741116 ... ...,[[-7.217436 1.612878 1.5321355 ... -5.0...,[[-2.924284 -2.4681082 0.16968031 ... -0.1...,[[-2.87291622 4.30007315 5.30276155 ... 2.3...,[[-2.6430743 0.9382878 -2.5283856 ... 0.3...,[[ 0.3239396 -1.652231 0.89375836 ... ...,...,[[-1.6366998 -0.9271893 0.5631146 ... -2.4...,[[-0.94835144 0.5831752 3.6347158 ... -4.9...,[[-0.92817354 -0.6032465 0.44023433 ... 4.2...,[[-4.5924788 1.4800639 -0.29908237 ... -2.4...,[[-1.1651084 -1.7349405 0.5957147 ... -4.0...,[[ 0.90246993 -4.2993584 2.4722567 ... 2.7...,[[-0.71411186 2.42939782 2.50810814 ... 2.9...,[[ 0.58169144 -2.5408762 -1.6491363 ... 3.2...,[[-0.71411186 2.4293978 2.5081081 ... 2.9...,[[ 0.19597623 1.5019727 4.5651073 ... 1.2...
2,4.9,4.4,4.7,3.8,4.5,4.8,4.8,4.7,3.2,4.2,...,3.1,4.2,4.5,4.4,3.6,4.5,4.6,4.2,4.5,3.7
3,ダイビングのため一人でカプセルルームに宿泊しました温泉は源泉かけながしなので循環のお湯にあり...,この価格で大きなお風呂を利用できるのはありがたかったです,食事は文句なしに美味しく満足できるものでした,プレステとニンテンドーのゲーム機があります,東伊豆でダイビングすることが多いのでまた利用するつもりです,温泉あり、夕飯朝食あり、広々とした部屋ありとってものんびり過ごせました！ありがとうございました！,この値段で、沢山の品揃え、アルコールまで飲んで充分満足しました,床で滑ってコーヒーをこぼしても、丁寧にすぐ綺麗に拭いて頂き、気持ち良かったです,卓球・カラオケ・麻雀・囲碁・・・全て無料とは驚きです,まるで昭和の時代にタイムスリップしたようでした,...,楽天スーパーSALEは、とてもいいプランでした,値段は安く、料理の種類も豊富で満足でした,部屋はやや古くても二間で使いやすかったです,ホテルの立地的な問題かもしれませんが、大浴場の風景はなく、まるでビルの谷間にあるイメージでした,宿泊代が安かったので、正直すべてにおいて期待していませんでした,が！！ホテル入口で駐車場の案内をしていたかた、駐車場で車の誘導をしていたかた、フロントのかた...,お部屋も広くて、お掃除も行き届いていましたし、あちこちにエアコンがあって暑がりな私たちも快適...,平日でしたが、土日のもっと宿泊客がいるときと変わらないものを出してくれているんだと思いました,お風呂もきれいでしたし、シャンプーが選べたりして楽しかったです,夏に私たち夫婦にとって大変つらいことがあり、元気になるための旅行でもありました


In [1]:
input_length = 25
input_data = []
for vector in data[0]:
    context_vectors = vector
    while(len(context_vectors)<input_length):
        context_vectors = np.append(context_vectors,[np.full(200,-1)], axis=0)
#     print(context_vectors)
    input_data.append(context_vectors)
output_data = data[1]

NameError: name 'data' is not defined

In [2]:
from sklearn.model_selection import train_test_split
input_data = np.array(input_data)
output_data = np.array(output_data)
# train_input_data, val_input_data, train_output_data, val_output_data = train_test_split(input_data,output_data,train_size=0.8, test_size=0.2,random_state=42)


# print(input_data.shape)
# print(output_data.shape)
# print(len(data[2]))

NameError: name 'np' is not defined

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers import Dropout
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras import backend as K 
np.random.seed(0)





N_train = int(len(input_data) * 0.8)
N_validation = len(input_data) - N_train

print(input_data.shape,output_data.shape, len(data[2]))
X_train, X_validation, Y_train, Y_validation, row_train, row_validation = \
    train_test_split(input_data, output_data, data[2], test_size=N_validation)


maxlen = 25
'''
モデル設定
'''
n_in = 200  
n_hidden = 512
n_out = 1


def weight_variable(shape, name=None):
    return np.random.normal(scale=.01, size=shape)

def root_mean_squared_error(y_true, y_pred): 
     return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
# early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1)

model = Sequential()
model.add(LSTM(n_hidden,
               kernel_initializer=weight_variable,
               input_shape=(maxlen, n_in)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
model.add(Dense(n_out, kernel_initializer=weight_variable))


model.summary()
# optimizer = Adam(lr=0.0010, beta_1=0.9, beta_2=0.999)
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
# model.compile(loss='mean_squared_error',optimizer=optimizer)
model.compile(loss=root_mean_squared_error,
              optimizer=optimizer)

/Users/uchidayuki/Python/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


AttributeError: 'list' object has no attribute 'shape'

In [4]:
'''
モデル学習
'''
epochs = 300
batch_size = 100

# model.fit(X_train, Y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_data=(X_validation, Y_validation),
#           callbacks=[early_stopping])

#earlystoppingなし
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_validation, Y_validation))


# model.fit(X_train, Y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_data=(X_validation, Y_validation))






NameError: name 'model' is not defined

In [5]:
# test_vectors = np.array(test_vectors)
y_pred = model.predict(X_train)
y_test_pred = model.predict(X_validation)
# for text, pred in zip(row_train, y_train_pred):
#     print(text, pred)

for text, pred, ans in zip(row_validation, y_test_pred, Y_validation):
    print(text, pred, ans)

NameError: name 'model' is not defined

In [6]:
'''
出力を用いて予測
'''
# 元データの最初の一部だけ切り出し
# model.predict()
# input_data[3:10].shape

test_text = [ "昨日、バイトがあって、お客さんがたくさん来ました","浅井さんは丁寧で、非常に優しい","前にここに来たときは、大雪だったので、あまり楽しめませんでした","宿は長野県にある","この旅館は人気がある","従業員の方々は非常に優しく、対応が丁寧です","従業員の方々が優しかったです","佐々木希が来たらしい","料理がとっても美味しくて、最高でした！","非常に景色と温泉が綺麗で、部屋の内装やアメニティが良く、非常に満足出来るものでした"]
test_wakati_text = []
for text in test_text:
    test_wakati_text.append(tokenize(text))

# test_wakati_text = ""
word_model = word2vec.Word2Vec.load("./word2vec/model/wiki.model")
test_vectors = []
for wakati_text in test_wakati_text:
    test_vector = []
    for char in m.removeStoplist(wakati_text, []).split(" "):
        try:
            test_vector.append(word_model[char])
        except KeyError:
            test_vector.append(unknown_token)
    test_vectors.append(np.array(test_vector))

# test_vectors = np.array(test_vectors)
input_length = 25
input_test_data = []
for test_vector in test_vectors:
    context_vectors = test_vector
    while(len(context_vectors)<input_length):
        context_vectors = np.append(context_vectors,[np.full(200,-1)], axis=0)
    input_test_data.append(context_vectors)
input_test_data = np.array(input_test_data)


test_predictions = model.predict(input_test_data)

for pred, text in zip(test_predictions, test_text):
    print(text, pred)

NameError: name 'tokenize' is not defined